In [1]:
import matplotlib,numpy,astropy
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astroquery.vizier import Vizier
from astroquery.ned import Ned
from astropy.table import QTable, join
from astropy import units
from astropy.coordinates import SkyCoord

In [2]:
c = 299792.458 * units.km/units.s #km/s
#HI line rest frequency
f0 = 1.420405751 * units.GHz #GHz

In [3]:
def logMHI_pred(B_c,D,B_c_err,D_err):
    '''
    Predicts HI mass of a galaxy given its B-band luminosity following Jones et al. 2018, A&A 609A, 17J.
    
    Inputs:
        B_c = Extinction and k-corrected B-band apparent magnitude.
        D = Source luminoisty distance [Mpc].
        B_c_err = Error in B_c.
        D_err = Error in D [Mpc].
        
    Outputs:
        logMHI = Logarithm of the predicted HI mass [Msol].
        logMHI_err = Error in logMHI.
    '''
    
    logD = numpy.log10(D)
    logD_err = D_err/(D*numpy.log(10.))
    
    logLB = 10. + 2.*numpy.log10(D) + 0.4*(4.88-B_c)
    
    logMHI = 0.94*logLB + 0.18
    
    logMHI_err = numpy.sqrt(0.2**2. + (0.94**2.)*((0.4*B_c_err)**2. + (2.*logD_err)**2.))
    
    return logMHI,logMHI_err

In [4]:
VLA_HCGs = [2, 7, 10, 15, 16, 19, 22, 23, 25, 26, 30, 31, 33, 37, 38, 40, 47, 48, 49, 54, 56, 57, 58, 59, 61, 62, 68, 71, 79, 88, 89, 90, 91, 92, 93, 95, 96, 97, 100]

#These distances were calculated manually using Cosmic Flows 3 model http://edd.ifa.hawaii.edu/CF3calculator/
#This calculator does not support automated queries
dists = [51.81, 51.64, 54.39, 94.25, 48.83, 51.95, 37.23, 63.46, 82.18, 127.30, 59.55, 52.03, 106.66, 95.08, 122.06, 97.21, 133.79, 36.79, 141.17, 26.71,
         117.03, 134.85, 83.60, 58.88, 59.03, 56.59, 36.23, 128.27, 60.25, 76.99, 122.83, 34.77, 91.81, 89.89, 66.11, 159.80, 119.51, 87.46, 69.30]

In [5]:
HCG_pos = Vizier(catalog='VII/213/groups', columns=['HCG', '_RAJ2000', '_DEJ2000'], row_limit=-1).query_constraints(HCG=','.join(numpy.array(VLA_HCGs,dtype='str')))[0]
HCG_pos.add_index('HCG')
HCG_z = Vizier(catalog='VII/213/dynamics', columns=['HCG', 'z'], row_limit=-1).query_constraints(HCG=','.join(numpy.array(VLA_HCGs,dtype='str')))[0]
HCG_z.add_index('HCG')

In [6]:
HCGs = join(HCG_pos,HCG_z,join_type='left')
HCGs['dist'] = dists * units.Mpc
HCGs.add_index('HCG')

HCGs.rename_column('_RAJ2000', 'ra')
HCGs.rename_column('_DEJ2000', 'dec')

In [7]:
#Unfortunately HCG 19 has no redshift in Hickson 1992
#Use NED redshift
HCGs.loc[19]['z'] = float(Ned.query_object("HCG19")['Redshift'])

In [8]:
HCGs['cz'] = c*HCGs['z']

#Convert heliocentric to "Local Sheet" velocities as in Kourkchi+2020, AJ, 159, 67
#These velocities were required for using the Cosmic Flows 3 calculator
#HCG_coords = SkyCoord(ra=HCGs['_RAJ2000'],dec=HCGs['_DEJ2000'],unit='deg')
#HCGs['l'] = HCG_coords.galactic.l.rad * units.rad
#HCGs['b'] = HCG_coords.galactic.b.rad * units.rad
#HCGs['v_ls'] = HCGs['cz'] - 26.*numpy.cos(HCGs['l'])*numpy.cos(HCGs['b']) + 317*numpy.sin(HCGs['l'])*numpy.cos(HCGs['b']) - 8.*numpy.sin(HCGs['b'])

In [10]:
#There is an error in the coordinates of HCG 48
HCGs.loc[48]['ra'] = 159.440201

In [12]:
HCG_mems = Vizier(catalog='VII/213/galaxies', columns=['HCG', 'm_HCG', '_RAJ2000', '_DEJ2000', 'MType', 'a', 'b', 'RVhel', 'BTmag', 'e_BTmag', 'BTc'], row_limit=-1).query_constraints(HCG=','.join(numpy.array(VLA_HCGs,dtype='str')))[0]

HCG_mems.rename_column('_RAJ2000', 'ra')
HCG_mems.rename_column('_DEJ2000', 'dec')
HCG_mems.rename_column('RVhel', 'Vel')

In [13]:
gals_to_remove = {2: ['d'], 19: ['d'], 22: ['d', 'e'], 23: ['e'], 25: ['c', 'e', 'g'], 31: ['d'], 38: ['d'], 48: ['c'], 59: ['e'], 61: ['b'], 
                  71: ['d'], 79: ['e'], 92: ['a'], 93: ['e']}

In [14]:
mem_names = []
for i in range(len(HCG_mems['HCG'])):
    mem_names.append('HCG'+str(HCG_mems[i]['HCG'])+str(HCG_mems[i]['m_HCG']))
HCG_mems['name'] = mem_names

In [15]:
mask = []
for HCG in gals_to_remove.keys():
    for mem in gals_to_remove[HCG]:
        for i in range(len(HCG_mems['HCG'])):
            if int(HCG_mems[i]['HCG']) == HCG and str(HCG_mems[i]['m_HCG']) == mem:
                mask.append(i)
                break
HCG_mems.remove_rows(mask)
HCG_mems.add_index('HCG')

In [16]:
#Values for added galaxies are from HyperLEDA which unfortunately is not currently searchable with astroquery
HCG_mems.add_row([16,'',32.5734980,-10.3214486,'SBab',None,None,3992,13.66,0.1,13.37,'NGC848'])
HCG_mems.add_row([16,'',32.2750604,-10.3202273,'Sc',None,None,3968,15.82,0.44,15.17,'PGC8210'])
HCG_mems.add_row([22,'',045.9313517,15.4852220,'S0',None,None,2628,14.60,0.39,14.32,'NGC1188'])
HCG_mems.add_row([90,'',330.6919058,-31.9554831,'Sbc',None,None,2568,16.18,0.19,15.27,'ESO466-47'])
HCG_mems.add_row([90,'',330.5661945,-31.7564831,'S0',None,None,2819,15.14,0.18,14.98,'ESO466-44'])
HCG_mems.add_row([90,'',330.6839214,-31.9910663,'S0a',None,None,2328,14.71,0.28,14.57,'ESO466-46'])

In [17]:
#Hickson used a different numberic scheme for Hubble type
#Here we will use the RC3 system

T_type = []
for M_type in HCG_mems['MType']:
    if "E" in M_type:
        T_type.append(-5)
    elif "S0" in M_type or "SB0" in M_type:
        T_type.append(0)
    elif "Sa" in M_type or "SBa" in M_type or "SB-" in M_type:
        T_type.append(1)
    elif "Sab" in M_type or "SBab" in M_type:
        T_type.append(2)
    elif "Sb" in M_type or "SBb" in M_type:
        T_type.append(3)
    elif "Sbc" in M_type or "SBbc" in M_type:
        T_type.append(4)
    elif "Sc" in M_type or "SBc" in M_type:
        T_type.append(5)
    elif "Scd" in M_type or "SBcd" in M_type:
        T_type.append(6)
    elif "Sd" in M_type or "SBd" in M_type:
        T_type.append(7)
    elif "Sdm" in M_type or "SBdm" in M_type:
        T_type.append(8)
    elif "Sm" in M_type or "SBm" in M_type:
        T_type.append(9)
    elif "Im" in M_type:
        T_type.append(10)
    elif "cI" in M_type:
        T_type.append(11)
    else:
        print('Case not matched:')
        print(M_type)
        
HCG_mems['T'] = T_type

In [18]:
HCG_mems = astropy.table.join(HCG_mems,HCGs['HCG','dist'],join_type='left')
HCG_mems.add_index('HCG')

In [19]:
HCG_mems['logMHI_pred'],HCG_mems['e_logMHI_pred'] = numpy.round(logMHI_pred(numpy.array(HCG_mems['BTc']),numpy.array(HCG_mems['dist']),HCG_mems['e_BTmag'],0.),decimals=2)

In [20]:
HCG_mems.pprint(max_lines=-1,max_width=-1)

HCG m_HCG      ra         dec     MType   a      b     Vel   BTmag e_BTmag  BTc     name    T   dist  logMHI_pred e_logMHI_pred
              deg         deg           arcsec arcsec km / s  mag    mag    mag                 Mpc                            
--- ----- ----------- ----------- ----- ------ ------ ------ ----- ------- ----- --------- --- ------ ----------- -------------
  2     a   7.8495541   8.4682225   SBd   40.5   19.7   4326 13.84    0.10 13.35     HCG2a   7  51.81        9.62           0.2
  2     b   7.8282520   8.4751805    cI   21.2   18.4   4366 14.63    0.10 14.39     HCG2b  11  51.81        9.23           0.2
  2     c   7.8723178   8.4006804   SBc   32.9   18.1   4235 14.60    0.10 14.15     HCG2c   5  51.81        9.32           0.2
  7     a   9.8065011   0.8636325    Sb   56.8   27.6   4210 13.46    0.10 12.98     HCG7a   3  51.64        9.75           0.2
  7     b   9.8251666   0.9127286   SB0   40.1   29.2   4238 14.02    0.20 13.74     HCG7b   0  51.64   

In [21]:
HI_pred = []
e_HI_pred = []
for HCG in HCGs['HCG']:
    HI_pred.append(numpy.log10(numpy.sum(10.**HCG_mems.loc[HCG]['logMHI_pred'])))
    e_HI_pred.append(numpy.sqrt(numpy.sum((HCG_mems.loc[HCG]['e_logMHI_pred']**2.)*10.**(2*HCG_mems.loc[HCG]['logMHI_pred']))/numpy.sum(10.**(2.*HCG_mems.loc[HCG]['logMHI_pred']))))
    
HCGs['logMHI_pred'] = numpy.round(HI_pred,decimals=2)
HCGs['e_logMHI_pred'] = numpy.round(e_HI_pred,decimals=2)

In [22]:
HCGs

HCG,ra,dec,z,dist,cz,logMHI_pred,e_logMHI_pred
,deg,deg,,Mpc,km / s,,
int16,float64,float64,float32,float64,float32,float64,float64
2,7.87517,8.43126,0.0144,51.81,4317.0117,9.9,0.2
7,9.84960,0.87818,0.0141,51.64,4227.0737,10.25,0.29
10,21.53076,34.69095,0.0161,54.39,4826.6587,10.31,0.2
15,31.91244,2.13827,0.0228,94.25,6835.2686,10.35,0.21
16,32.38881,-10.16298,0.0132,48.83,3957.2605,10.35,0.21
19,40.68807,-12.41181,0.0140,51.95,4197.0947,9.68,0.21
22,45.88027,-15.67570,0.0090,37.23,2698.132,9.96,0.22
23,46.77701,-9.58547,0.0161,63.46,4826.6587,9.82,0.21


In [23]:
HCG_mems = HCG_mems['HCG','m_HCG','name','ra','dec','MType','T','a','b','Vel','BTmag','e_BTmag','BTc','dist','logMHI_pred','e_logMHI_pred']

In [24]:
HCG_mems.write('../output/HCG_members.vo',format='votable',overwrite=True)
HCG_mems.write('../output/HCG_members.fits',format='fits',overwrite=True)
HCG_mems.write('../output/HCG_members.ascii',format='ascii.fixed_width_two_line',overwrite=True)

In [25]:
HCGs.write('../output/HCGs.vo',format='votable',overwrite=True)
HCGs.write('../output/HCGs.fits',format='fits',overwrite=True)
HCGs.write('../output/HCGs.ascii',format='ascii.fixed_width_two_line',overwrite=True)